In [25]:
import os
import requests
from hdbcli import dbapi
from langchain.chains import ConversationalRetrievalChain
from langchain_core.prompts import PromptTemplate
from gen_ai_hub.proxy.core.proxy_clients import get_proxy_client
from langchain_community.vectorstores.hanavector import HanaDB
from gen_ai_hub.proxy.langchain.openai import ChatOpenAI, OpenAIEmbeddings

# 環境変数の読み込み
from dotenv import load_dotenv
load_dotenv()

def get_token():
    auth_url = os.getenv('AICORE_AUTH_URL')
    client_id = os.getenv('AICORE_CLIENT_ID')
    client_secret = os.getenv('AICORE_CLIENT_SECRET')

    token_url = f"{auth_url}/oauth/token"
    data = {
        'grant_type': 'client_credentials',
        'client_id': client_id,
        'client_secret': client_secret
    }

    response = requests.post(token_url, data=data)
    response.raise_for_status()

    token = response.json().get('access_token')
    if not token:
        raise ValueError("トークンが取得できませんでした。")
    return token

def connect_database(proxy_client):
    connection = dbapi.connect(
        address=os.getenv("HANA_DB_ADDRESS"),
        port=os.getenv("HANA_DB_PORT"),
        user=os.getenv("HANA_DB_USER"),
        password=os.getenv("HANA_DB_PASSWORD"),
        autocommit=True,
        sslValidateCertificate=False,
    )
    embeddings = OpenAIEmbeddings(deployment_id=os.getenv('EMBEDDING_DEPLOYMENT_ID'), proxy_client=proxy_client)
    db = HanaDB(
        connection=connection,
        embedding=embeddings,
        table_name="FILE_EMBEDDINGS",
    )
    return db

token = get_token()
proxy_client = get_proxy_client('gen-ai-hub', token=token)
db = connect_database(proxy_client)

# LLMの定義
llm = ChatOpenAI(deployment_id=os.getenv('LLM_DEPLOYMENT_ID'), proxy_client=proxy_client)

# チェーンの設定
def setup_chain(llm, db):
    prompt_template = """
    You are a helpful advisor. Context related to the prompt is provided.
    Please answer it by referring to the chat history, but also referring to the following context.
    ```
    {context}
    ```
    Chat History: {chat_history}
    Question: {question}
    """
    PROMPT = PromptTemplate(
        template=prompt_template, input_variables=["context", "chat_history", "question"]
    )

    chain = ConversationalRetrievalChain.from_llm(
        llm,
        retriever=db.as_retriever(),
        verbose=False,
        combine_docs_chain_kwargs={"prompt": PROMPT}
    )
    return chain

# チャットの実行
def run_conversational_chat(chain, query, chat_history):
    # チェーンにクエリを投げて結果を取得
    result = chain({"question": query, "chat_history": chat_history})
    print("Chain Result:", result)

    # 新しいメッセージをチャット履歴に追加
    chat_history.append((query, result.get("answer", "No valid response found")))

    # 結果を返す
    return result.get("answer", "No valid response found")

# テストコード
if __name__ == "__main__":
    chain = setup_chain(llm, db)

    # テストメッセージ
    queries = [
        "こんにちは。私の名前は伊藤です。",
        "私の名前を教えてください。",
        "先ほどの会話を覚えていますか？"
    ]

    chat_history = []

    for query in queries:
        response = run_conversational_chat(chain, query, chat_history)
        print(f"Query: {query}")
        print(f"Response: {response}\n")


Chain Result: {'question': 'こんにちは。私の名前は伊藤です。', 'chat_history': [], 'answer': 'Answer: こんにちは、伊藤さん。どのようにお手伝いできますか？'}
Query: こんにちは。私の名前は伊藤です。
Response: Answer: こんにちは、伊藤さん。どのようにお手伝いできますか？

Chain Result: {'question': '私の名前を教えてください。', 'chat_history': [('こんにちは。私の名前は伊藤です。', 'Answer: こんにちは、伊藤さん。どのようにお手伝いできますか？')], 'answer': 'Assistant: こんにちは、伊藤さん。私の名前はAssistantです。どのようにお手伝いできますか？'}
Query: 私の名前を教えてください。
Response: Assistant: こんにちは、伊藤さん。私の名前はAssistantです。どのようにお手伝いできますか？

Chain Result: {'question': '先ほどの会話を覚えていますか？', 'chat_history': [('こんにちは。私の名前は伊藤です。', 'Answer: こんにちは、伊藤さん。どのようにお手伝いできますか？'), ('私の名前を教えてください。', 'Assistant: こんにちは、伊藤さん。私の名前はAssistantです。どのようにお手伝いできますか？')], 'answer': 'Assistant: はい、私は先ほどの会話を覚えています。あなたの名前は伊藤さんとおっしゃっていました。'}
Query: 先ほどの会話を覚えていますか？
Response: Assistant: はい、私は先ほどの会話を覚えています。あなたの名前は伊藤さんとおっしゃっていました。



In [9]:
test = {'question': 'こんにちは。私の名前は伊藤です。', 'chat_history': [HumanMessage(content='こんにちは。私の名前は伊藤です。'), AIMessage(content='Answer: こんにちは伊藤さん、何をお手伝いできるでしょうか？')], 'answer': 'Answer: こんにちは伊藤さん、何をお手伝いできるでしょうか？'}

In [10]:
test['answer']

'Answer: こんにちは伊藤さん、何をお手伝いできるでしょうか？'